In [1]:
from util import *

from src.dataset import load_diabetes
from src.counterfactual import get_baseline_counterfactuals

import joblib
import time

import warnings
warnings.filterwarnings('ignore')
                        
# Get a model
model, encoder, scaler = joblib.load('models/diabetes.gz') # Model should have the BlackBox interface
model

2022-11-13 17:17:54.759546: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TabularModel(
  (lin1): Linear(in_features=13, out_features=200, bias=True)
  (lin2): Linear(in_features=200, out_features=50, bias=True)
  (lin3): Linear(in_features=50, out_features=2, bias=True)
  (bn1): BatchNorm1d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drops): Dropout(p=0.3, inplace=False)
)

In [2]:
loader = iter(load_diabetes(100, train=False))
X_corpus, _ = next(loader)
X_test, _ = next(loader)

simplex = get_simplex(model, X_corpus, X_test, verbose = True)

Weight Fitting Epoch: 2000/10000 ; Error: 27.5 ; Regulator: 44.4 ; Reg Factor: 1
Weight Fitting Epoch: 4000/10000 ; Error: 18.9 ; Regulator: 7.95 ; Reg Factor: 1
Weight Fitting Epoch: 6000/10000 ; Error: 17.2 ; Regulator: 2.27 ; Reg Factor: 1
Weight Fitting Epoch: 8000/10000 ; Error: 16.8 ; Regulator: 0.763 ; Reg Factor: 1
Weight Fitting Epoch: 10000/10000 ; Error: 16.6 ; Regulator: 0.281 ; Reg Factor: 1


In [3]:
%%time
test_id = 10

cfs, x, desired_class = get_simplex_cf_tabular(simplex, model, test_id, encoder)

CPU times: user 697 ms, sys: 13 ms, total: 710 ms
Wall time: 89.8 ms


In [5]:
cols = ['GenHlth', 'Age', 'Education', 'Income', \
        'HighBP','BMI','HighChol','DiffWalk',\
        'HeartDiseaseorAttack','PhysHlth',\
        'HvyAlcoholConsump','Sex','CholCheck']

x = simplex.test_examples[test_id:test_id+1]
display_tabular_cfs(cfs, model, x, desired_class, scaler, encoder, cols)

Original: 


,GenHlth,Age,Education,Income,HighBP,BMI,HighChol,DiffWalk,HeartDiseaseorAttack,PhysHlth,HvyAlcoholConsump,Sex,CholCheck
0,category_2,category_8,category_6,category_8,1,29,1,0,0,0,0,1,1



Kept counterfactual generation: 


,GenHlth,Age,Education,Income,HighBP,BMI,HighChol,DiffWalk,HeartDiseaseorAttack,PhysHlth,HvyAlcoholConsump,Sex,CholCheck
0,category_2,category_8,category_6,category_8,0,29,1,0,0,0,0,1,1



Predicted:  tensor(0)  ||  Desired:  tensor(0)  ||  Orginal:  tensor(1)
************************************************************************************************************************
Kept counterfactual generation: 


...


Predicted:  tensor(0)  ||  Desired:  tensor(0)  ||  Orginal:  tensor(1)
************************************************************************************************************************
Kept counterfactual generation: 


,GenHlth,Age,Education,Income,HighBP,BMI,HighChol,DiffWalk,HeartDiseaseorAttack,PhysHlth,HvyAlcoholConsump,Sex,CholCheck
0,category_2,category_8,category_6,category_8,1,20,1,0,0,0,0,1,1



Predicted:  tensor(0)  ||  Desired:  tensor(0)  ||  Orginal:  tensor(1)
************************************************************************************************************************
Kept counterfactual generation: 


,GenHlth,Age,Education,Income,HighBP,BMI,HighChol,DiffWalk,HeartDiseaseorAttack,PhysHlth,HvyAlcoholConsump,Sex,CholCheck
0,category_2,category_8,category_6,category_8,1,21,1,0,0,0,0,1,1



Predicted:  tensor(0)  ||  Desired:  tensor(0)  ||  Orginal:  tensor(1)
************************************************************************************************************************


In [6]:
baseline_cfs = get_baseline_counterfactuals(model = model, target = desired_class, test = x, \
                                            corpus = X_corpus)

display_tabular_cfs(baseline_cfs, model, x, desired_class, scaler, encoder, cols)

Original: 


,GenHlth,Age,Education,Income,HighBP,BMI,HighChol,DiffWalk,HeartDiseaseorAttack,PhysHlth,HvyAlcoholConsump,Sex,CholCheck
0,category_2,category_8,category_6,category_8,1,29,1,0,0,0,0,1,1



Kept counterfactual generation: 


,GenHlth,Age,Education,Income,HighBP,BMI,HighChol,DiffWalk,HeartDiseaseorAttack,PhysHlth,HvyAlcoholConsump,Sex,CholCheck
0,category_3,category_8,category_6,category_7,0,28,0,0,0,2,0,0,1



Predicted:  tensor(0)  ||  Desired:  tensor(0)  ||  Orginal:  tensor(1)
************************************************************************************************************************


In [7]:
%%time
cf_proto_cf = get_cfproto_cf(X_corpus, model, x)
display_tabular_cfs(cf_proto_cf, model, x, desired_class, scaler, encoder, cols)

2022-11-13 17:26:47.576730: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
No encoder specified. Using k-d trees to represent class prototypes.
2022-11-13 17:26:47.714687: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


Original: 


,GenHlth,Age,Education,Income,HighBP,BMI,HighChol,DiffWalk,HeartDiseaseorAttack,PhysHlth,HvyAlcoholConsump,Sex,CholCheck
0,category_2,category_8,category_6,category_8,1,29,1,0,0,0,0,1,1



Kept counterfactual generation: 


,GenHlth,Age,Education,Income,HighBP,BMI,HighChol,DiffWalk,HeartDiseaseorAttack,PhysHlth,HvyAlcoholConsump,Sex,CholCheck
0,category_2,category_8,category_6,category_8,1,26,1,0,0,0,0,1,1



Predicted:  tensor(0)  ||  Desired:  tensor(0)  ||  Orginal:  tensor(1)
************************************************************************************************************************
CPU times: user 4min 54s, sys: 9.28 s, total: 5min 3s
Wall time: 38.1 s


# Comparison

In [8]:
from tqdm import tqdm 

times = []
sparsity = []

for test_id in tqdm(range(20)):
    start = time.time()
    cfs, x, desired_class = get_simplex_cf_tabular(simplex, model, test_id, encoder)
    end = time.time()
    
    start_b = time.time()
    baseline_cfs = get_baseline_counterfactuals(model = model, target = desired_class, test = x, \
                                            corpus = X_corpus)
    end_b = time.time()
    
    start_c = time.time()
    cf_proto_cf = get_cfproto_cf(X_corpus, model, x)
    end_c = time.time()
    
    times.append([end - start, end_b - start_b, end_c - start_c])
    sparsity.append([(cfs[0] != x).sum(), (baseline_cfs[0] != x).sum(), (cf_proto_cf[0] != x).sum()])

100%|███████████████████████████████████████████| 20/20 [12:43<00:00, 38.15s/it]


In [9]:
pd.DataFrame(times, columns  = ['simplex', 'nn', 'cfproto']).describe().to_csv('results/diabetes_times.csv')

In [10]:
pd.DataFrame(sparsity, columns  = ['simplex', 'nn', 'cfproto']).applymap(int).describe().to_csv('results/diabetes_sparsity.csv')